### Local Binary Patterns + XGBoost optimization

Вибір оптимальних параметрів для вирішення задач класифікації являється важливою та ресурсо-затратною задачею. В даній роботі буде запропонована одна із можливих реалізацій алгоритму підбору значеть конфігурації для XGBoost.

Щоб підібрати оптимальні параметри для XGBoost, можна використати метод Grid Search для пошуку найкращих комбінацій значень гіперпараметрів.

Пропонований підхід

    Grid Search із перебором параметрів:
        Cтворюємо сітку значень для n_estimators, max_depth, і eta.
        Перевіримо кожну комбінацію цих параметрів шляхом повного навчання і оцінки класифікатора на тестових даних.
        Також розглянемо розширення сітки додатковими параметрами для LBP.

    Оцінка моделі:
        Під час Grid Search використовуємо метрики (наприклад, accuracy) для вибору найкращої моделі.
        Результати будемо зберігати для порівняння.

    Реалізація:
        До існуючої реалізації класифікатора додати функцію, яка виконуватиме Grid Search.
        Організувати збереження результатів у вигляді таблиці або списку для аналізу.
        Змінити функцію train_xgboost_classifier, щоб вона підтримувала гіперпараметри, що надходять як аргументи.


#### Імпортуємо необхідні бібліотеки

З головного тут XGBoost та local_binary_pattern

In [181]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import random
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from skimage.feature import local_binary_pattern
import time

In [182]:
import warnings
# Вимкнути всі попередження для видаленн зайвої інформації з консолі
warnings.filterwarnings('ignore')

#### Функції виділення ознак


LBP фокусується на текстурі, тому додавання інформації про колір може значно розширити можливості класифікатора.
В даному дослідженні до LBP ознак буде додано інформацію значення пікселя в каналі RGB.

In [203]:
def prepare_data_with_colored_mask(image, mask, color_to_class, pixel_sample_percentage):
    """
    Вибирає координати та мітки із кольорової маски для тренування та генерує маску тренувальних пікселів.
    
    Parameters:
        image: Вхідне RGB-зображення.
        mask: Вхідна кольорова маска (BGR), яка вказує на мітки класів.
        color_to_class: Словник, що зв'язує кольори маски з індексами класів.
        pixel_sample_percentage: Відсоток пікселів для вибору в тренувальну підвибірку.
    
    Returns:
        sampled_coordinates: Список координат [(y, x)] вибраних пікселів.
        sampled_labels: Список міток класів для вибраних пікселів.
        temp_training_mask: Проміжна маска, що містить лише використовувані у тренуванні пікселі.
    """
    coordinates = []
    labels = []
    
    # Ініціалізуємо порожню маску для візуалізації тренувальних пікселів
    training_mask = np.ones_like(mask) * 255  # Маска спочатку біла (не використані пікселі)
    
    # Проходимо через пікселі вхідної маски
    for y in range(mask.shape[0]):
        for x in range(mask.shape[1]):
            b, g, r = mask[y, x]  # OpenCV використовує формат BGR
            color = (r, g, b)
            if color in color_to_class:
                coordinates.append((y, x))
                labels.append(color_to_class[color])
                # Копіюємо кольори з оригінальної маски в проміжну маску
                training_mask[y, x] = mask[y, x]
            elif color == (255, 255, 255):  # Ігноруємо білі пікселі
                continue

    # Перемішуємо та відбираємо пікселі
    combined = list(zip(coordinates, labels))
    random.shuffle(combined)
    coordinates, labels = zip(*combined)
    
    # Вибираємо підмножину пікселів для тренування
    sample_size = int(len(coordinates) * pixel_sample_percentage)
    sampled_coordinates = coordinates[:sample_size]
    sampled_labels = labels[:sample_size]
    
    # Змінюємо training_mask так, щоб залишилася тільки підвибірка
    temp_training_mask = np.ones_like(mask) * 255  # Створюємо нову маску з білим фоном
    for (y, x) in sampled_coordinates:
        temp_training_mask[y, x] = mask[y, x]  # Копіюємо тільки координати вибірки
    
    # Повертаємо результати
    return sampled_coordinates, sampled_labels, temp_training_mask

###  Реалізація Grid Search

In [204]:
def grid_search_xgboost(features, labels, param_grid):
    """
    Виконує Grid Search для підбору оптимальних параметрів XGBoost з вимірюванням часу виконання.
    Покращено для відображення ТОП-5 найкращих параметрів.

    Parameters:
        features: Матриця ознак для тренування.
        labels: Відповідні класи для ознак.
        param_grid: Словник параметрів для Grid Search:
            {
                'n_estimators': [100, 200, 300],
                'max_depth': [4, 6, 8],
                'eta': [0.05, 0.1, 0.3]
            }

    Returns:
        best_params: Найкращі параметри після Grid Search.
        top_params: ТОП-5 найкращих параметрів (з найбільшим accuracy).
        results: Результати для всіх комбінацій параметрів.
    """
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    import xgboost as xgb
    import time

    # Розбиваємо дані на тренувальні й тестові
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)
    results = []  # Список для результатів (accuracy та час для кожної комбінації)

    # Перебір усіх комбінацій параметрів
    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for eta in param_grid['eta']:
                start_time = time.perf_counter()  # Відлік часу для цієї комбінації

                # Ініціалізація моделі з поточними параметрами
                model = xgb.XGBClassifier(
                    objective='multi:softmax',
                    num_class=4,
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    eta=eta
                )
                
                # Навчання моделі
                model.fit(X_train, y_train)
                
                # Передбачення тестових даних
                y_pred = model.predict(X_test)
                
                # Оцінка точності
                accuracy = accuracy_score(y_test, y_pred)
                
                # Обрахунок часу виконання
                elapsed_time = time.perf_counter() - start_time
                
                # Збереження результату
                results.append({
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'eta': float(eta),  # Перетворюємо eta в float
                    'accuracy': float(accuracy),  # Перетворюємо accuracy в float
                    'time_elapsed': elapsed_time  # Час виконання в секундах
                })
                print(f"Evaluated params: n_estimators={n_estimators}, max_depth={max_depth}, eta={eta}, "
                      f"accuracy={accuracy:.4f}, time_elapsed={elapsed_time:.2f} seconds")

    # Сортуємо список результатів за значенням 'accuracy' (у спадному порядку)
    sorted_results = sorted(results, key=lambda x: x['accuracy'], reverse=True)

    # Пошук найкращих параметрів за accuracy
    best_params = sorted_results[0]

    # Топ-5 найкращих параметрів за accuracy
    top_params = sorted_results[:5]
    
    # Виведення найкращих параметрів
    print(f"\nBest Parameters: {best_params}")
    
    # Виведення ТОП-5 параметрів у зручному форматі
    print("\nTop 5 Parameters:")
    print(f"{'Rank':<5} {'n_estimators':<12} {'max_depth':<10} {'eta':<6} {'accuracy':<8} {'time(s)':<8}")
    for i, params in enumerate(top_params, start=1):
        print(f"{i:<5} {params['n_estimators']:<12} {params['max_depth']:<10} {params['eta']:<6.2f} "
              f"{params['accuracy']:<8.4f} {params['time_elapsed']:<8.2f}")

    return best_params, top_params, results

### Оптимізація по часу

Дана функція покликана надати оцінку моделі враховуючи два параметри Accuracy та Time. Для цього введено додатковий коефіцієнт який визначає важливість часу необхідного для тренування та роботи моделі.

In [205]:
def select_best_by_accuracy_time(results, lambda_weight=1.0, n_top=1):
    """
    Обирає оптимальні параметри, комбінуючи точність та час виконання, і відображає ТОП N найкращих.

    Parameters:
        results: Список результатів Grid Search (із ключами 'accuracy' і 'time_elapsed').
        lambda_weight: Вага часу у метриці: Score = Accuracy - lambda_weight * Time.
        n_top: Кількість найкращих параметрів, які потрібно повернути (за замовчуванням 1).

    Returns:
        top_results: ТОП N результатів із Grid Search, відсортовані у спадному порядку за 'score'.
    """
    # Обрахунок скорингу для кожного результату
    for result in results:
        result['score'] = result['accuracy'] - lambda_weight * result['time_elapsed']
    
    # Сортуємо список за значенням 'score' у спадному порядку
    sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)
    
    # Повертаємо ТОП N результатів
    top_results = sorted_results[:n_top]

    # Виведення ТОП N результатів у форматі таблиці
    print(f"\nTop {n_top} Parameters that take to account time:")
    print(f"{'Rank':<5} {'n_estimators':<12} {'max_depth':<10} {'eta':<6} {'accuracy':<8} {'time(s)':<8}")
    for i, params in enumerate(top_results, start=1):
        print(f"{i:<5} {params['n_estimators']:<12} {params['max_depth']:<10} {params['eta']:<6.2f} "
              f"{params['accuracy']:<8.4f} {params['time_elapsed']:<8.2f}")
    
    return top_results

### Візуалізація отриманих результатів

Інколи візуалізація дозволяє отимати краще розуміння про задіяні параметри. Дана функція це спроба візуалізації точок на площині 'max_depth' та 'eta', яка показує значення accuracy. Візуалізація будується для кожного значення n_estimators окремо.

In [206]:
def plot_grid_results_3d_scatter_with_lines(grid_results, n_estimators_values):
    """
    Побудова 3D-розташування точок (scatter plot) для значень accuracy,
    та вертикальних ліній, що опускаються з точок на площину XY,
    для кожного значення n_estimators.

    Parameters:
        grid_results: Список результатів Grid Search (містить 'n_estimators', 'max_depth', 'eta', 'accuracy').
        n_estimators_values: Список значень n_estimators для окремих графіків.

    Returns:
        None
    """
    for n_estimators in n_estimators_values:
        # Фільтруємо результати для поточного n_estimators
        filtered_results = [res for res in grid_results if res['n_estimators'] == n_estimators]

        # Список значень для X, Y, Z
        eta_values = [res['eta'] for res in filtered_results]
        max_depth_values = [res['max_depth'] for res in filtered_results]
        accuracy_values = [res['accuracy'] for res in filtered_results]

        # Генеруємо кольори залежно від значень accuracy
        cmap = get_cmap('viridis')
        norm = plt.Normalize(min(accuracy_values), max(accuracy_values))
        colors = cmap(norm(accuracy_values))

        # Побудова 3D-графіка
        fig = plt.figure(figsize=(14, 8))
        ax = fig.add_subplot(111, projection='3d')

        # Побудова точок
        sc = ax.scatter(eta_values, max_depth_values, accuracy_values, c=colors, s=50, cmap='viridis', edgecolor='k')

        # Додавання вертикальних ліній до площини XY
        for x, y, z in zip(eta_values, max_depth_values, accuracy_values):
            ax.plot([x, x], [y, y], [0, z], color='black', linestyle='dashed', linewidth=0.7)

        # Налаштування осей
        ax.set_title(f"Accuracy for n_estimators={n_estimators}")
        ax.set_xlabel("eta (learning_rate)")
        ax.set_ylabel("max_depth")
        ax.set_zlabel("Accuracy")

        # Додаємо кольорову шкалу
        mappable = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
        mappable.set_array(accuracy_values)
        fig.colorbar(mappable, ax=ax, shrink=0.6, aspect=8, label='Accuracy')

        # Показуємо графік
        plt.tight_layout()
        plt.show()

In [207]:
def extract_pixel_features_rgb_with_lbp_and_colors(image, coordinates, radius=1, n_points=8, method='uniform'):
    """
    Extract LBP features and RGB values for each pixel coordinate.
    
    Parameters:
        image: RGB input image.
        coordinates: List of pixel coordinates [(y1, x1), (y2, x2), ...].
        radius: Radius for the LBP descriptor.
        n_points: Number of sampling points for the LBP descriptor.
        method: LBP calculation method ('uniform', 'default', etc.).
    
    Returns:
        np.array: Feature vectors for each pixel.
    """
    features = []
    b_channel, g_channel, r_channel = cv2.split(image)

    for coord in coordinates:
        y, x = coord
        lbp_features = []
        for channel in [b_channel, g_channel, r_channel]:
            patch = channel[max(0, y - radius):min(channel.shape[0], y + radius + 1),
                            max(0, x - radius):min(channel.shape[1], x + radius + 1)]
            if patch.shape[0] < 2 * radius + 1 or patch.shape[1] < 2 * radius + 1:
                patch = cv2.resize(patch, (2 * radius + 1, 2 * radius + 1))
            lbp = local_binary_pattern(patch, P=n_points, R=radius, method=method)
            lbp_features.extend(lbp.flatten())

        rgb_values = [r_channel[y, x], g_channel[y, x], b_channel[y, x]]
        feature_vector = np.concatenate((lbp_features, rgb_values))
        features.append(feature_vector)

    return np.array(features)

### Навчання моделі

Використаємо стандартний підхід до навчання, коли навчальні дані розбиваються на дві частини: тренувальні дані та перевірочні дані.
Навідміну від попереднього дослдіження LBP+Color+XGBoost per-pixel Sentinel дана функція здатна приймати більше параметрів на вхід, що дозволяє запустити цикл з різними параметрами для їх оцінки.

In [208]:
def train_xgboost_classifier(features, labels, n_estimators=300, max_depth=6, eta=0.3):
    """
    Train an XGBoost classifier.
    
    Parameters:
        features: Feature matrix (N x M, where M is the feature size).
        labels: Corresponding labels (N x 1).
    
    Returns:
        Trained XGBoost model.
    """
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

    model = xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=4,
        n_estimators=n_estimators,
        max_depth=max_depth,
        eta=eta
    )
    
    model.fit(X_train, y_train)
    
    # Evaluate model performance
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    
    return model

### Функція класифікація

В даній функції реалізована можливість оцінки моделі на зображеннях відмінних від тих які використовувались для її навчання та перевірки.

In [209]:
def classify_entire_image(image, model, radius=1, n_points=8, class_to_color=None, expected_size=None, use_color=True, use_lbp=True):
    """
    Classify each pixel in the input image using the trained model with support for LBP, RGB, or LBP+RGB features.

    Parameters:
        image: Input RGB image.
        model: Trained XGBoost model or any compatible classifier.
        radius: Radius for the LBP descriptor.
        n_points: Number of sampling points for the LBP descriptor.
        class_to_color: Mapping of class indices to RGB colors.
        expected_size: Expected size of feature vectors for inference (must match training).
        use_color: Whether to include RGB color values in the feature vector (True to include RGB).
        use_lbp: Whether to include LBP features in the feature vector (True to include LBP).

    Returns:
        full_mask: Output classification mask (RGB image).
    """
    height, width = image.shape[:2]
    full_mask = np.zeros((height, width, 3), dtype=np.uint8)

    # Loop through each pixel and classify
    for y in range(height):
        for x in range(width):
            feature_vector = []  # This will store both LBP and/or RGB features based on settings

            # Extract LBP features if enabled
            if use_lbp:
                lbp_features = []
                for channel in cv2.split(image):
                    # Extract local patch around the pixel
                    patch = channel[max(0, y - radius):min(channel.shape[0], y + radius + 1),
                                    max(0, x - radius):min(channel.shape[1], x + radius + 1)]
                    if patch.shape[0] < 2 * radius + 1 or patch.shape[1] < 2 * radius + 1:
                        patch = cv2.resize(patch, (2 * radius + 1, 2 * radius + 1))
                    # Compute LBP for the local patch
                    lbp = local_binary_pattern(patch, P=n_points, R=radius, method='uniform')
                    lbp_features.extend(lbp.flatten())  # Append LBP features for the channel
                feature_vector.extend(lbp_features)  # Add LBP features to the feature vector

            # Extract color features if enabled
            if use_color:
                rgb_values = [image[y, x, 2], image[y, x, 1], image[y, x, 0]]  # RGB order
                feature_vector.extend(rgb_values)  # Add RGB values to the feature vector

            # Convert feature vector to np.array
            feature_vector = np.array(feature_vector)

            # Ensure feature vector size matches the expected size for the model
            if expected_size is None:
                # Retrieve the expected size from the model
                expected_size = len(model.feature_names_in_)
            if feature_vector.shape[0] != expected_size:
                if feature_vector.shape[0] < expected_size:
                    feature_vector = np.pad(feature_vector, (0, expected_size - feature_vector.shape[0]), mode='constant')
                elif feature_vector.shape[0] > expected_size:
                    feature_vector = feature_vector[:expected_size]  # Truncate features if too large

            # Run prediction for the pixel
            prediction = model.predict(np.array([feature_vector]))[0]
            # Map the prediction to the corresponding color
            full_mask[y, x] = class_to_color[prediction]

    return full_mask

### Окрема функція для перевірки моделі

Дана функція оцінює точність класифікація відповідно до наданої маски. Це надає можливість провести чистий експеремент коли дані яку були використані для тренування та перевірки моделі, та дані які будуть використовуватись для оцінки моделі відрізняються.

In [210]:

def evaluate_classification_results(resulting_mask, eval_mask, color_to_class):
    """
    Оцінює точність класифікації на основі повної маски для оцінки.

    Parameters:
        resulting_mask: Класифікована маска (BGR).
        eval_mask: Маска для оцінки точності (BGR).
        color_to_class: Словник, що встановлює відповідність кольорів класам.

    Returns:
        accuracy: Точність класифікації.
        report: Повний звіт класифікації.
    """
    eval_labels = []
    predicted_labels = []
    
    # Білий колір (зазвичай ігноровані пікселі)
    ignored_color = (255, 255, 255)  # Білий у форматі BGR

    # Проходимо через пікселі оцінкової маски
    for y in range(eval_mask.shape[0]):
        for x in range(eval_mask.shape[1]):
            eval_color = tuple(eval_mask[y, x])  # BGR формат
            predicted_color = tuple(resulting_mask[y, x])  # BGR формат

            # Ігнорувати пікселі білого кольору
            if eval_color == ignored_color:
                continue

            # Перевіряємо, чи кольори присутні в словнику для оцінки
            if eval_color in color_to_class:
                eval_labels.append(color_to_class[eval_color])
            else:
                print(f"Warning: eval_color {eval_color} не знайдено в color_to_class.")
                continue  # Пропускаємо, якщо eval_color немає в словнику
                
            if predicted_color in color_to_class:
                predicted_labels.append(color_to_class[predicted_color])
            else:
                print(f"Warning: predicted_color {predicted_color} не знайдено в color_to_class.")
                continue  # Пропускаємо, якщо predicted_color немає в словнику

    # Якщо після всіх перевірок немає пікселів для обчислення, повернемо нульові значення
    if len(eval_labels) == 0 or len(predicted_labels) == 0:
        print("Warning: У обраній області немає відповідних пікселів для оцінки.")
        return 0.0, "No data available for evaluation."

    # Використовуємо sklearn для оцінки точності
    accuracy = accuracy_score(eval_labels, predicted_labels)
    report = classification_report(eval_labels, predicted_labels, zero_division=0)
    
    return accuracy, report

### Головна функція

В даній функції використовується підхід (LBP + Color) + XGBoost та зображення Sentinel data. Функція застосовує Grid Search для перебору параметрів та знаходження найоптимальніших значень для XGBoost та LBP. В рамках дослідження проведемо оцінку оптимальної конфігурації беручи до уваги рекомендованя значення LBP та змінюючи лише конфігурацію XGBoost. А також порівняємо з результататми які отримаємо для  сітки яка буде вразовувати зміни не лише для XGBosst а  XGBoost + LBP.

Оцінимо роботу на основі двох зображень Sentinel data (SS1 та SS2).

### (LBP + Color) + XGBoost

In [217]:
if __name__ == "__main__":
    # Підрахунок часу для всього процесу
    total_start_time = time.perf_counter()

    # --- Обробка першого зображеня ---
    print("==================================")
    print("Evaluation SS1")
    print("==================================")
    process_start_time = time.perf_counter()
    
    # Шляхи до зображень   
    image_path = "Images/sentinel/SS1/SS1.bmp"
    eval_image_path = "Images/sentinel/SS1/SS1.bmp"  # Шумлене зображення
    
    # Шляхи до масок
    train_mask_path = "Images/sentinel/SS1/Samples1.bmp"  # Маска для тренування
    eval_mask_path = "Images/sentinel/SS1/Etalon1.bmp"  # Маска для оцінки точності класифікації (повна)
    
    # Завантажуємо зображення та маски
    rgb_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    eval_image = cv2.imread(eval_image_path, cv2.IMREAD_COLOR)  # Зашумлене зображення
    train_mask = cv2.imread(train_mask_path, cv2.IMREAD_COLOR)  # Маска для тренування
    eval_mask = cv2.imread(eval_mask_path, cv2.IMREAD_COLOR)  # Маска для тренування
    eval_mask_rgb = cv2.cvtColor(eval_mask, cv2.COLOR_BGR2RGB) # Маска для оцінки точності

    # Встановлюємо колір-клас відповідність:
    color_to_class = {
        (0, 0, 255): 0,    # Синій - Вода
        (0, 255, 0): 1,    # Зелений - Рослинність
        (0, 0, 0): 2,      # Чорний - Відкритий ґрунт
        (255, 255, 0): 3   # Жовтий - Урбанізація
    }

    class_to_color = {v: k for k, v in color_to_class.items()}  # Зворотня відповідність для класифікованого результату

    # === Тренування моделі ===
    sampled_coordinates, sampled_labels, training_mask = prepare_data_with_colored_mask(
        rgb_image, train_mask, color_to_class, pixel_sample_percentage=1.0
    )

    # # Візуалізуємо вибрану тренувальну маску
    # training_mask_rgb = cv2.cvtColor(training_mask, cv2.COLOR_BGR2RGB)  # Переводимо в RGB для matplotlib
    # plt.figure(figsize=(8, 8))
    # plt.imshow(training_mask_rgb)
    # plt.axis('off')
    # plt.title("Training Mask")
    # plt.show()
    # cv2.imwrite("Images/temp/training_mask.png", training_mask)  # Зберігаємо тренувальну маску для аналізу

    # LBP + RGB
    features = extract_pixel_features_rgb_with_lbp_and_colors(
        rgb_image, sampled_coordinates, radius=2, n_points=16
    )

    # --- Пошук найкращих параметрів ---
    # param_grid = {
    #     'n_estimators': [100, 200, 300, 500],  # Кількість дерев для навчання
    #     'max_depth': [4, 6, 8, 10, 12],        # Глибина дерев
    #     'eta': [0.01, 0.05, 0.1, 0.2, 0.3, 0.5] # Швидкість навчання
    # }
    param_grid = {
        'n_estimators': [100, 200, 300, 500],
        'max_depth': [4, 6, 8],
        'eta': [0.05, 0.1, 0.3]
    }

    best_params, top_5_params, grid_results = grid_search_xgboost(features, sampled_labels, param_grid)

    # # Значення n_estimators, для яких хочемо побудувати графіки
    # #n_estimators_values = param_grid['n_estimators']    
    # n_estimators_values = [100, 200, 300]
    
    # # Побудова точкових 3D-графіків
    # plot_grid_results_3d_scatter_with_lines(grid_results, n_estimators_values)
    
    # --- Вибір оптимальної комбінації точність/час ---
    lambda_weight = 0.1  # Регулювання впливу часу на вибір метрики (залежно від вимог задачі)
    optimal_results = select_best_by_accuracy_time(grid_results, lambda_weight=1.0, n_top=5)

    # Тренуємо модель XGBoost
    model = train_xgboost_classifier(
        features=features,
        labels=sampled_labels,
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        eta=best_params['eta']
    )
       
    # === Класифікація первірочного зображення ===   

    # # Візуалізуємо вибрану перевірочну маску
    # plt.figure(figsize=(8, 8))
    # plt.imshow(eval_mask_rgb)
    # plt.axis('off')
    # plt.title("Eval Mask")
    # plt.show()
    # cv2.imwrite("Images/temp/eval_mask.png", training_mask)  # Зберігаємо тренувальну маску для аналізу
    
    eval_resulting_mask = classify_entire_image(
        eval_image, model, radius=2, n_points=16, class_to_color=class_to_color, expected_size=features.shape[1], use_color=True, use_lbp=True
    )
    eval_resulting_mask_rgb = cv2.cvtColor(eval_resulting_mask, cv2.COLOR_BGR2RGB)  # Формат RGB
    # plt.figure(figsize=(8, 8))
    # plt.imshow(eval_resulting_mask)
    # plt.axis('off')
    # plt.title("Resulting Mask with LBP (Eval Image)")
    # plt.show()
    # cv2.imwrite("Images/temp/eval_result_colored_mask_lbp.png", eval_resulting_mask_rgb)  # Зберігаємо результат
    
    # === Оцінка точності на основі повної маски ===
    eval_resulting_mask_bgr = cv2.cvtColor(eval_resulting_mask, cv2.COLOR_RGB2BGR)

    # Виведення унікальних кольорів у масці
    unique_colors_eval, counts_eval = np.unique(eval_mask.reshape(-1, 3), axis=0, return_counts=True)
    print("Unique colors in eval_mask (BGR):", dict(zip(map(tuple, unique_colors_eval), counts_eval)))
    
    eval_accuracy, eval_report = evaluate_classification_results(
        eval_resulting_mask, eval_mask_rgb, color_to_class
    )
    
    # Виведення результатів
    print(f"Evaluation Accuracy on Eval Image: {eval_accuracy * 100:.2f}%")
    print("Evaluation Classification Report:")
    print(eval_report)

    # Показуємо час для першого зображення
    process_end_time = time.perf_counter()
    print(f"Time taken for first image pair processing: {process_end_time - process_start_time:.2f} seconds")

    # --- Обробка другого зображеня ---
    print("==================================")
    print("Evaluation SS2")
    print("==================================")

        # Шляхи до зображень   
    image_path = "Images/sentinel/SS2/SS2.bmp"
    eval_image_path = "Images/sentinel/SS2/SS2.bmp"  # Шумлене зображення
    
    # Шляхи до масок
    train_mask_path = "Images/sentinel/SS2/Samples2.bmp"  # Маска для тренування
    eval_mask_path = "Images/sentinel/SS2/Etalon2.bmp"  # Маска для оцінки точності класифікації (повна)
    
    # Завантажуємо зображення та маски
    rgb_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    eval_image = cv2.imread(eval_image_path, cv2.IMREAD_COLOR)  # Зашумлене зображення
    train_mask = cv2.imread(train_mask_path, cv2.IMREAD_COLOR)  # Маска для тренування
    eval_mask = cv2.imread(eval_mask_path, cv2.IMREAD_COLOR)  # Маска для тренування
    eval_mask_rgb = cv2.cvtColor(eval_mask, cv2.COLOR_BGR2RGB) # Маска для оцінки точності

    # Встановлюємо колір-клас відповідність:
    color_to_class = {
        (0, 0, 255): 0,    # Синій - Вода
        (0, 255, 0): 1,    # Зелений - Рослинність
        (0, 0, 0): 2,      # Чорний - Відкритий ґрунт
        (255, 255, 0): 3   # Жовтий - Урбанізація
    }

    class_to_color = {v: k for k, v in color_to_class.items()}  # Зворотня відповідність для класифікованого результату

    # === Тренування моделі ===
    sampled_coordinates, sampled_labels, training_mask = prepare_data_with_colored_mask(
        rgb_image, train_mask, color_to_class, pixel_sample_percentage=1.0
    )

    # # Візуалізуємо вибрану тренувальну маску
    # training_mask_rgb = cv2.cvtColor(training_mask, cv2.COLOR_BGR2RGB)  # Переводимо в RGB для matplotlib
    # plt.figure(figsize=(8, 8))
    # plt.imshow(training_mask_rgb)
    # plt.axis('off')
    # plt.title("Training Mask")
    # plt.show()
    # cv2.imwrite("Images/temp/training_mask.png", training_mask)  # Зберігаємо тренувальну маску для аналізу

    # LBP + RGB
    features = extract_pixel_features_rgb_with_lbp_and_colors(
        rgb_image, sampled_coordinates, radius=2, n_points=16
    )

    # --- Пошук найкращих параметрів ---
    # param_grid = {
    #     'n_estimators': [100, 200, 300, 500],  # Кількість дерев для навчання
    #     'max_depth': [4, 6, 8, 10, 12],        # Глибина дерев
    #     'eta': [0.01, 0.05, 0.1, 0.2, 0.3, 0.5] # Швидкість навчання
    # }
    param_grid = {
        'n_estimators': [100, 200, 300, 500],
        'max_depth': [4, 6, 8],
        'eta': [0.05, 0.1, 0.3]
    }

    best_params, top_5_params, grid_results = grid_search_xgboost(features, sampled_labels, param_grid)

    # # Значення n_estimators, для яких хочемо побудувати графіки
    # #n_estimators_values = param_grid['n_estimators']    
    # n_estimators_values = [100, 200, 300]
    
    # # Побудова точкових 3D-графіків
    # plot_grid_results_3d_scatter_with_lines(grid_results, n_estimators_values)
    
    # --- Вибір оптимальної комбінації точність/час ---
    lambda_weight = 0.1  # Регулювання впливу часу на вибір метрики (залежно від вимог задачі)
    optimal_results = select_best_by_accuracy_time(grid_results, lambda_weight=1.0, n_top=5)

    # Тренуємо модель XGBoost
    model = train_xgboost_classifier(
        features=features,
        labels=sampled_labels,
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        eta=best_params['eta']
    )
       
    # === Класифікація первірочного зображення ===   

    # # Візуалізуємо вибрану перевірочну маску
    # plt.figure(figsize=(8, 8))
    # plt.imshow(eval_mask_rgb)
    # plt.axis('off')
    # plt.title("Eval Mask")
    # plt.show()
    # cv2.imwrite("Images/temp/eval_mask.png", training_mask)  # Зберігаємо тренувальну маску для аналізу
    
    eval_resulting_mask = classify_entire_image(
        eval_image, model, radius=2, n_points=16, class_to_color=class_to_color, expected_size=features.shape[1], use_color=True, use_lbp=True
    )
    eval_resulting_mask_rgb = cv2.cvtColor(eval_resulting_mask, cv2.COLOR_BGR2RGB)  # Формат RGB
    # plt.figure(figsize=(8, 8))
    # plt.imshow(eval_resulting_mask)
    # plt.axis('off')
    # plt.title("Resulting Mask with LBP (Eval Image)")
    # plt.show()
    # cv2.imwrite("Images/temp/eval_result_colored_mask_lbp.png", eval_resulting_mask_rgb)  # Зберігаємо результат
    
    # === Оцінка точності на основі повної маски ===
    eval_resulting_mask_bgr = cv2.cvtColor(eval_resulting_mask, cv2.COLOR_RGB2BGR)

    # Виведення унікальних кольорів у масці
    unique_colors_eval, counts_eval = np.unique(eval_mask.reshape(-1, 3), axis=0, return_counts=True)
    print("Unique colors in eval_mask (BGR):", dict(zip(map(tuple, unique_colors_eval), counts_eval)))
    
    eval_accuracy, eval_report = evaluate_classification_results(
        eval_resulting_mask, eval_mask_rgb, color_to_class
    )
    
    # Виведення результатів
    print(f"Evaluation Accuracy on Eval Image: {eval_accuracy * 100:.2f}%")
    print("Evaluation Classification Report:")
    print(eval_report)

    # Підрахунок загального часу роботи програми
    total_end_time = time.perf_counter()
    print(f"Total time taken: {total_end_time - total_start_time:.2f} seconds")

Evaluation SS1
Evaluated params: n_estimators=100, max_depth=4, eta=0.05, accuracy=0.9721, time_elapsed=1.16 seconds
Evaluated params: n_estimators=100, max_depth=4, eta=0.1, accuracy=0.9768, time_elapsed=1.19 seconds
Evaluated params: n_estimators=100, max_depth=4, eta=0.3, accuracy=0.9822, time_elapsed=1.21 seconds
Evaluated params: n_estimators=100, max_depth=6, eta=0.05, accuracy=0.9776, time_elapsed=1.43 seconds
Evaluated params: n_estimators=100, max_depth=6, eta=0.1, accuracy=0.9817, time_elapsed=1.53 seconds
Evaluated params: n_estimators=100, max_depth=6, eta=0.3, accuracy=0.9842, time_elapsed=1.48 seconds
Evaluated params: n_estimators=100, max_depth=8, eta=0.05, accuracy=0.9791, time_elapsed=1.78 seconds
Evaluated params: n_estimators=100, max_depth=8, eta=0.1, accuracy=0.9822, time_elapsed=1.83 seconds
Evaluated params: n_estimators=100, max_depth=8, eta=0.3, accuracy=0.9845, time_elapsed=1.91 seconds
Evaluated params: n_estimators=200, max_depth=4, eta=0.05, accuracy=0.977

###  Аналіз отриманих значень

З отриманих значень слідує що найоптимальнішою конфігурацією XGBoost з запропонованих параметрів для зображення SS1 являється наступна:

Top 5 Parameters:
Rank  n_estimators max_depth  eta    accuracy time(s) 
1     500          8          0.30   0.9860   7.61    
2     300          8          0.30   0.9858   4.98    
3     200          8          0.30   0.9855   3.33    
4     500          6          0.10   0.9852   7.25    
5     500          6          0.30   0.9850   6.96  

Для SS2:
Top 5 Parameters:
Rank  n_estimators max_depth  eta    accuracy time(s) 
1     500          4          0.30   0.9578   3.63    
2     300          4          0.30   0.9575   2.31    
3     200          4          0.30   0.9567   1.57    
4     500          6          0.30   0.9565   4.76    
5     500          8          0.30   0.9562   5.28   


В даному досліді не стоїть задача оптимзації моделі по часу, а лише пошук параметрів які покажуть найвищу точність.

З отриманих значень можна зробити наступні висновки. Збільшення кількості дерев та глибини узагальному призводить до ускладнення обчислень та збільшення часу необхідного для підготовки моделі. Модель показала найкращі результати при високих значеннях n_estimators та max_depth. При цьому зменшення швидкості навчання, тобто eta - незначно погіршило результати.

### Оновлена функція Grid Search

Дана функція дозволяє враховувати додатково конфігурацію LBP.

In [228]:
def grid_search_xgboost(labels, param_grid, rgb_image, sampled_coordinates):
    """
    Виконує Grid Search для підбору оптимальних параметрів XGBoost, включаючи параметри 'radius' та 'n_points',
    для функції extract_pixel_features_rgb_with_lbp_and_colors.

    Parameters:
        labels: Відповідні класи для ознак.
        param_grid: Словник параметрів для Grid Search:
            {
                'n_estimators': [100, 200, 300],
                'max_depth': [4, 6, 8],
                'eta': [0.05, 0.1, 0.3],
                'radius': [1, 2, 3],
                'n_points': [8, 16, 24]
            }
        rgb_image: Вхідне RGB-зображення.
        sampled_coordinates: Координати пікселів для витягу ознак.

    Returns:
        best_params: Найкращі параметри після Grid Search.
        top_params: ТОП-5 найкращих параметрів (з найбільшим accuracy).
        results: Результати для всіх комбінацій параметрів.
    """
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    import xgboost as xgb
    import time

    results = []  # Список для результатів (accuracy та час для кожної комбінації)

    # Перебір усіх комбінацій параметрів
    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for eta in param_grid['eta']:
                for radius in param_grid['radius']:
                    for n_points in param_grid['n_points']:
                        start_time = time.perf_counter()  # Відлік часу для цієї комбінації

                        # === Витягуємо ознаки з новими параметрами radius і n_points ===
                        features = extract_pixel_features_rgb_with_lbp_and_colors(
                            rgb_image, sampled_coordinates, radius=radius, n_points=n_points
                        )

                        # Розбиваємо дані на тренувальні й тестові
                        X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

                        # Ініціалізація моделі з поточними параметрами
                        model = xgb.XGBClassifier(
                            objective='multi:softmax',
                            num_class=4,
                            n_estimators=n_estimators,
                            max_depth=max_depth,
                            eta=eta
                        )
                        
                        # Навчання моделі
                        model.fit(X_train, y_train)
                        
                        # Передбачення тестових даних
                        y_pred = model.predict(X_test)
                        
                        # Оцінка точності
                        accuracy = accuracy_score(y_test, y_pred)
                        
                        # Обрахунок часу виконання
                        elapsed_time = time.perf_counter() - start_time
                        
                        # Збереження результату
                        results.append({
                            'n_estimators': n_estimators,
                            'max_depth': max_depth,
                            'eta': float(eta),  # Перетворюємо eta в float
                            'radius': radius,    # Додаємо radius
                            'n_points': n_points, # Додаємо n_points
                            'accuracy': float(accuracy),  # Перетворюємо accuracy в float
                            'time_elapsed': elapsed_time  # Час виконання в секундах
                        })
                        print(f"Evaluated params: n_estimators={n_estimators}, max_depth={max_depth}, eta={eta}, "
                              f"radius={radius}, n_points={n_points}, accuracy={accuracy:.4f}, time_elapsed={elapsed_time:.2f} seconds")

    # Сортуємо список результатів за значенням 'accuracy' (у спадному порядку)
    sorted_results = sorted(results, key=lambda x: x['accuracy'], reverse=True)

    # Пошук найкращих параметрів за accuracy
    best_params = sorted_results[0]

    # Топ-5 найкращих параметрів за accuracy
    top_params = sorted_results[:10]
    
    # Виведення найкращих параметрів
    print(f"\nBest Parameters: {best_params}")

    # Виведення ТОП-5 параметрів у зручному форматі
    print("\nTop 5 Parameters:")
    print(f"{'Rank':<5} {'n_estimators':<12} {'max_depth':<10} {'eta':<6} {'radius':<8} {'n_points':<10} {'accuracy':<8} {'time(s)':<8}")
    for i, params in enumerate(top_params, start=1):
        print(f"{i:<5} {params['n_estimators']:<12} {params['max_depth']:<10} {params['eta']:<6.2f} "
              f"{params['radius']:<8} {params['n_points']:<10} {params['accuracy']:<8.4f} {params['time_elapsed']:<8.2f}")

    return best_params, top_params, results

### Головна функція комплексного підбору

Дана функція дозволяє отримати значення точності класифікації для різних конфігурація XGBoost та LBP.

In [229]:
if __name__ == "__main__":
    # Підрахунок часу для всього процесу
    total_start_time = time.perf_counter()

    # --- Обробка першого зображеня ---
    print("==================================")
    print("Evaluation SS1")
    print("==================================")
    process_start_time = time.perf_counter()
    
    # Шляхи до зображень   
    image_path = "Images/sentinel/SS1/SS1.bmp"
    eval_image_path = "Images/sentinel/SS1/SS1.bmp"  # Шумлене зображення
    
    # Шляхи до масок
    train_mask_path = "Images/sentinel/SS1/Samples1.bmp"  # Маска для тренування
    eval_mask_path = "Images/sentinel/SS1/Etalon1.bmp"  # Маска для оцінки точності класифікації (повна)
    
    # Завантажуємо зображення та маски
    rgb_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    eval_image = cv2.imread(eval_image_path, cv2.IMREAD_COLOR)  # Зашумлене зображення
    train_mask = cv2.imread(train_mask_path, cv2.IMREAD_COLOR)  # Маска для тренування
    eval_mask = cv2.imread(eval_mask_path, cv2.IMREAD_COLOR)  # Маска для тренування
    eval_mask_rgb = cv2.cvtColor(eval_mask, cv2.COLOR_BGR2RGB) # Маска для оцінки точності

    # Встановлюємо колір-клас відповідність:
    color_to_class = {
        (0, 0, 255): 0,    # Синій - Вода
        (0, 255, 0): 1,    # Зелений - Рослинність
        (0, 0, 0): 2,      # Чорний - Відкритий ґрунт
        (255, 255, 0): 3   # Жовтий - Урбанізація
    }

    class_to_color = {v: k for k, v in color_to_class.items()}  # Зворотня відповідність для класифікованого результату

    # === Тренування моделі ===
    sampled_coordinates, sampled_labels, training_mask = prepare_data_with_colored_mask(
        rgb_image, train_mask, color_to_class, pixel_sample_percentage=1.0
    )

    # # Візуалізуємо вибрану тренувальну маску
    # training_mask_rgb = cv2.cvtColor(training_mask, cv2.COLOR_BGR2RGB)  # Переводимо в RGB для matplotlib
    # plt.figure(figsize=(8, 8))
    # plt.imshow(training_mask_rgb)
    # plt.axis('off')
    # plt.title("Training Mask")
    # plt.show()
    # cv2.imwrite("Images/temp/training_mask.png", training_mask)  # Зберігаємо тренувальну маску для аналізу

    # LBP + RGB
    features = extract_pixel_features_rgb_with_lbp_and_colors(
        rgb_image, sampled_coordinates, radius=2, n_points=16
    )

    # --- Пошук найкращих параметрів ---
    # param_grid = {
    #     'n_estimators': [100, 200, 300, 500],  # Кількість дерев для навчання
    #     'max_depth': [4, 6, 8, 10, 12],        # Глибина дерев
    #     'eta': [0.01, 0.05, 0.1, 0.2, 0.3, 0.5] # Швидкість навчання
    # }
    param_grid = {
            'n_estimators': [100, 200, 300, 500],
            'max_depth': [4, 6, 8],
            'eta': [0.05, 0.1, 0.3],
            'radius': [1, 2, 3],
            'n_points': [8, 16, 24]
    }
    
    # param_grid = {
    #         'n_estimators': [100],
    #         'max_depth': [4, 6],
    #         'eta': [0.05, 0.1],
    #         'radius': [1, 2],
    #         'n_points': [8, 16]
    # }

    best_params, top_5_params, grid_results = grid_search_xgboost(sampled_labels, param_grid, rgb_image, sampled_coordinates)

    # # Значення n_estimators, для яких хочемо побудувати графіки
    # #n_estimators_values = param_grid['n_estimators']    
    # n_estimators_values = [100, 200, 300]
    
    # # Побудова точкових 3D-графіків
    # plot_grid_results_3d_scatter_with_lines(grid_results, n_estimators_values)
    
    # --- Вибір оптимальної комбінації точність/час ---
    lambda_weight = 0.1  # Регулювання впливу часу на вибір метрики (залежно від вимог задачі)
    optimal_results = select_best_by_accuracy_time(grid_results, lambda_weight=1.0, n_top=10)

    # Тренуємо модель XGBoost
    model = train_xgboost_classifier(
        features=features,
        labels=sampled_labels,
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        eta=best_params['eta']
    )
       
    # === Класифікація первірочного зображення ===   

    # # Візуалізуємо вибрану перевірочну маску
    # plt.figure(figsize=(8, 8))
    # plt.imshow(eval_mask_rgb)
    # plt.axis('off')
    # plt.title("Eval Mask")
    # plt.show()
    # cv2.imwrite("Images/temp/eval_mask.png", training_mask)  # Зберігаємо тренувальну маску для аналізу
    
    eval_resulting_mask = classify_entire_image(
        eval_image, model, radius=best_params['radius'], n_points=best_params['n_points'], class_to_color=class_to_color, expected_size=features.shape[1], use_color=True, use_lbp=True
    )
    eval_resulting_mask_rgb = cv2.cvtColor(eval_resulting_mask, cv2.COLOR_BGR2RGB)  # Формат RGB
    # plt.figure(figsize=(8, 8))
    # plt.imshow(eval_resulting_mask)
    # plt.axis('off')
    # plt.title("Resulting Mask with LBP (Eval Image)")
    # plt.show()
    # cv2.imwrite("Images/temp/eval_result_colored_mask_lbp.png", eval_resulting_mask_rgb)  # Зберігаємо результат
    
    # === Оцінка точності на основі повної маски ===
    eval_resulting_mask_bgr = cv2.cvtColor(eval_resulting_mask, cv2.COLOR_RGB2BGR)

    # Виведення унікальних кольорів у масці
    unique_colors_eval, counts_eval = np.unique(eval_mask.reshape(-1, 3), axis=0, return_counts=True)
    print("Unique colors in eval_mask (BGR):", dict(zip(map(tuple, unique_colors_eval), counts_eval)))
    
    eval_accuracy, eval_report = evaluate_classification_results(
        eval_resulting_mask, eval_mask_rgb, color_to_class
    )
    
    # Виведення результатів
    print(f"Evaluation Accuracy on Eval Image: {eval_accuracy * 100:.2f}%")
    print("Evaluation Classification Report:")
    print(eval_report)

    # Показуємо час для першого зображення
    process_end_time = time.perf_counter()
    print(f"Time taken for first image pair processing: {process_end_time - process_start_time:.2f} seconds")

    # --- Обробка другого зображеня ---
    print("==================================")
    print("Evaluation SS2")
    print("==================================")

        # Шляхи до зображень   
    image_path = "Images/sentinel/SS2/SS2.bmp"
    eval_image_path = "Images/sentinel/SS2/SS2.bmp"  # Шумлене зображення
    
    # Шляхи до масок
    train_mask_path = "Images/sentinel/SS2/Samples2.bmp"  # Маска для тренування
    eval_mask_path = "Images/sentinel/SS2/Etalon2.bmp"  # Маска для оцінки точності класифікації (повна)
    
    # Завантажуємо зображення та маски
    rgb_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    eval_image = cv2.imread(eval_image_path, cv2.IMREAD_COLOR)  # Зашумлене зображення
    train_mask = cv2.imread(train_mask_path, cv2.IMREAD_COLOR)  # Маска для тренування
    eval_mask = cv2.imread(eval_mask_path, cv2.IMREAD_COLOR)  # Маска для тренування
    eval_mask_rgb = cv2.cvtColor(eval_mask, cv2.COLOR_BGR2RGB) # Маска для оцінки точності

    # Встановлюємо колір-клас відповідність:
    color_to_class = {
        (0, 0, 255): 0,    # Синій - Вода
        (0, 255, 0): 1,    # Зелений - Рослинність
        (0, 0, 0): 2,      # Чорний - Відкритий ґрунт
        (255, 255, 0): 3   # Жовтий - Урбанізація
    }

    class_to_color = {v: k for k, v in color_to_class.items()}  # Зворотня відповідність для класифікованого результату

    # === Тренування моделі ===
    sampled_coordinates, sampled_labels, training_mask = prepare_data_with_colored_mask(
        rgb_image, train_mask, color_to_class, pixel_sample_percentage=1.0
    )

    # # Візуалізуємо вибрану тренувальну маску
    # training_mask_rgb = cv2.cvtColor(training_mask, cv2.COLOR_BGR2RGB)  # Переводимо в RGB для matplotlib
    # plt.figure(figsize=(8, 8))
    # plt.imshow(training_mask_rgb)
    # plt.axis('off')
    # plt.title("Training Mask")
    # plt.show()
    # cv2.imwrite("Images/temp/training_mask.png", training_mask)  # Зберігаємо тренувальну маску для аналізу

    # LBP + RGB
    features = extract_pixel_features_rgb_with_lbp_and_colors(
        rgb_image, sampled_coordinates, radius=2, n_points=16
    )

    # --- Пошук найкращих параметрів ---
    # param_grid = {
    #     'n_estimators': [100, 200, 300, 500],  # Кількість дерев для навчання
    #     'max_depth': [4, 6, 8, 10, 12],        # Глибина дерев
    #     'eta': [0.01, 0.05, 0.1, 0.2, 0.3, 0.5] # Швидкість навчання
    # }
    # param_grid = {
    #         'n_estimators': [100],
    #         'max_depth': [4, 6],
    #         'eta': [0.05, 0.1],
    #         'radius': [1, 2],
    #         'n_points': [8, 16]
    # }
    param_grid = {
            'n_estimators': [100, 200, 300, 500],
            'max_depth': [4, 6, 8],
            'eta': [0.05, 0.1, 0.3],
            'radius': [1, 2, 3],
            'n_points': [8, 16, 24]
    }

    # best_params, top_5_params, grid_results = grid_search_xgboost(features, sampled_labels, param_grid)
    best_params, top_5_params, grid_results = grid_search_xgboost(sampled_labels, param_grid, rgb_image, sampled_coordinates)

    # # Значення n_estimators, для яких хочемо побудувати графіки
    # #n_estimators_values = param_grid['n_estimators']    
    # n_estimators_values = [100, 200, 300]
    
    # # Побудова точкових 3D-графіків
    # plot_grid_results_3d_scatter_with_lines(grid_results, n_estimators_values)
    
    # --- Вибір оптимальної комбінації точність/час ---
    lambda_weight = 0.1  # Регулювання впливу часу на вибір метрики (залежно від вимог задачі)
    optimal_results = select_best_by_accuracy_time(grid_results, lambda_weight=1.0, n_top=10)

    # Тренуємо модель XGBoost
    model = train_xgboost_classifier(
        features=features,
        labels=sampled_labels,
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        eta=best_params['eta']
    )
       
    # === Класифікація первірочного зображення ===   

    # # Візуалізуємо вибрану перевірочну маску
    # plt.figure(figsize=(8, 8))
    # plt.imshow(eval_mask_rgb)
    # plt.axis('off')
    # plt.title("Eval Mask")
    # plt.show()
    # cv2.imwrite("Images/temp/eval_mask.png", training_mask)  # Зберігаємо тренувальну маску для аналізу
    
    eval_resulting_mask = classify_entire_image(
        eval_image, model, radius=best_params['radius'], n_points=best_params['n_points'], class_to_color=class_to_color, expected_size=features.shape[1], use_color=True, use_lbp=True
    )
    eval_resulting_mask_rgb = cv2.cvtColor(eval_resulting_mask, cv2.COLOR_BGR2RGB)  # Формат RGB
    # plt.figure(figsize=(8, 8))
    # plt.imshow(eval_resulting_mask)
    # plt.axis('off')
    # plt.title("Resulting Mask with LBP (Eval Image)")
    # plt.show()
    # cv2.imwrite("Images/temp/eval_result_colored_mask_lbp.png", eval_resulting_mask_rgb)  # Зберігаємо результат
    
    # === Оцінка точності на основі повної маски ===
    eval_resulting_mask_bgr = cv2.cvtColor(eval_resulting_mask, cv2.COLOR_RGB2BGR)

    # Виведення унікальних кольорів у масці
    unique_colors_eval, counts_eval = np.unique(eval_mask.reshape(-1, 3), axis=0, return_counts=True)
    print("Unique colors in eval_mask (BGR):", dict(zip(map(tuple, unique_colors_eval), counts_eval)))
    
    eval_accuracy, eval_report = evaluate_classification_results(
        eval_resulting_mask, eval_mask_rgb, color_to_class
    )
    
    # Виведення результатів
    print(f"Evaluation Accuracy on Eval Image: {eval_accuracy * 100:.2f}%")
    print("Evaluation Classification Report:")
    print(eval_report)

    # Підрахунок загального часу роботи програми
    total_end_time = time.perf_counter()
    print(f"Total time taken: {total_end_time - total_start_time:.2f} seconds")

Evaluation SS1
Evaluated params: n_estimators=100, max_depth=4, eta=0.05, radius=1, n_points=8, accuracy=0.9687, time_elapsed=6.16 seconds
Evaluated params: n_estimators=100, max_depth=4, eta=0.05, radius=1, n_points=16, accuracy=0.9706, time_elapsed=6.13 seconds
Evaluated params: n_estimators=100, max_depth=4, eta=0.05, radius=1, n_points=24, accuracy=0.9706, time_elapsed=6.26 seconds
Evaluated params: n_estimators=100, max_depth=4, eta=0.05, radius=2, n_points=8, accuracy=0.9695, time_elapsed=7.01 seconds
Evaluated params: n_estimators=100, max_depth=4, eta=0.05, radius=2, n_points=16, accuracy=0.9704, time_elapsed=7.03 seconds
Evaluated params: n_estimators=100, max_depth=4, eta=0.05, radius=2, n_points=24, accuracy=0.9706, time_elapsed=7.44 seconds
Evaluated params: n_estimators=100, max_depth=4, eta=0.05, radius=3, n_points=8, accuracy=0.9725, time_elapsed=7.83 seconds
Evaluated params: n_estimators=100, max_depth=4, eta=0.05, radius=3, n_points=16, accuracy=0.9719, time_elapsed=8

---

## Підсумок

Скрипт складається з кількох модулів, які відповідають за різні етапи роботи класифікатора. Основний класифікатор працює на основі **LBP (Local Binary Patterns)** для текстурних ознак і кольорових ознак (RGB). На кожному пікселі (попіксельно) витягуються ознаки для навчання моделі **XGBoost**, яка потім використовується для класифікації кожного пікселя на новому зображенні.

---

### Основні функції в скрипті

1. **`prepare_data_with_colored_mask`**:
   Ця функція забезпечує підготовку даних для навчання класифікатора. Вона:
   - Вибирає координати пікселів із маски (`mask`), які позначені кольором.
   - Перетворює кольори маски у класи відповідно до заданого словника (`color_to_class`).
   - Вибірково бере частину пікселів (визначається параметром `pixel_sample_percentage`), що дозволяє навчати класифікатор на меншій і більш репрезентативній вибірці даних.

   **Конфігурування:**
   - **`pixel_sample_percentage`:**
     - Якщо класи сильно незбалансовані, збільшіть відсоток (наприклад, `0.7` або `1.0`), щоб захопити більше даних для рідкісних класів.
     - Зменшіть його для дуже великих масок і зображень, щоб уникнути високої обчислювальної складності.

   **Приклад виклику:**
   ```python
   sampled_coordinates, sampled_labels = prepare_data_with_colored_mask(
       rgb_image, mask, color_to_class, pixel_sample_percentage=0.5
   )
   ```

---

2. **`extract_pixel_features_rgb_with_lbp_and_colors`**:
   Ця функція витягує ознаки для кожного пікселя:
   - **LBP:** Текстурні ознаки, що відображають зміни яскравості в локальній області навколо пікселя (`radius`/`n_points`).
   - **RGB:** Кольорові значення пікселя (R, G, B).

   Усі ці компоненти об'єднуються в один вектор ознак, який використовується для навчання XGBoost.

   **Конфігурування:**
   - **`radius`:**
     - Радіус LBP визначає розмір локального контексту. Малий радіус (`1`–`2`) працює для текстур із дрібними деталями, великий радіус (`3`–`5`) — для більш широких текстур.
   - **`n_points`:**
     - Визначає кількість точок (пікселів) для LBP. Типово `n_points = 8 * radius`.
   - **`method`:**
     - Метод розрахунку LBP. Типово `"uniform"` для більш стабільних ознак.

   **Приклад виклику:**
   ```python
   features = extract_pixel_features_rgb_with_lbp_and_colors(
       rgb_image, sampled_coordinates, radius=2, n_points=16
   )
   ```

---

3. **`train_xgboost_classifier`**:
   Ця функція тренує модель XGBoost на основі витягнутих ознак (`features`) і класів пікселів (`labels`).

   **Конфігурування:**
   - **Hyperparameters in XGBoost (настроювані параметри):**
     - **`n_estimators` (кількість дерев):** Збільшіть до `200–300` для покращення точності моделі.
     - **`max_depth` (глибина дерев):** Баланс між деталізацією моделі та її схильністю до переобучення. Оптимально `4–8`.
     - **`eta` (learning_rate):** Зменште до `0.1` або `0.05` для більш плавного навчання моделі.

   **Приклад виклику:**
   ```python
   model = train_xgboost_classifier(features, sampled_labels)
   ```

---

4. **`classify_entire_image`**:
   Ця функція класифікує кожен піксель нового зображення, використовуючи натреновану модель XGBoost.

   - Витягує ознаки за допомогою LBP і RGB значень.
   - Передбачає клас кожного пікселя та генерує фінальну відобрачену маску у форматі RGB (або BGR).

   **Конфігурування:**
   - Параметри витягнення ознак (`radius`, `n_points`) повинні співпадати з навчальними.
   - Якщо виникають проблеми з розміром вектора ознак, використовуйте автоматичний підгін:
     ```python
     expected_size = features.shape[1]  # Отримане з навчальних даних
     feature_vector = np.pad(feature_vector, (0, expected_size - feature_vector.shape[0]), mode='constant')
     ```

   **Приклад виклику:**
   ```python
   resulting_mask = classify_entire_image(
       rgb_image, model, radius=2, n_points=16, class_to_color=class_to_color, expected_size=expected_size
   )
   ```

---

### Загальна структура функцій

##### **Основний алгоритм**:
```python
if __name__ == "__main__":
    # 1. Завантажуємо зображення та маску.
    rgb_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    mask = cv2.imread(mask_path, cv2.IMREAD_COLOR)

    # 2. Готуємо координати та класи для навчання за маскою.
    sampled_coordinates, sampled_labels = prepare_data_with_colored_mask(
        rgb_image, mask, color_to_class, pixel_sample_percentage=0.5
    )

    # 3. Витягуємо ознаки LBP та кольору.
    features = extract_pixel_features_rgb_with_lbp_and_colors(
        rgb_image, sampled_coordinates, radius=2, n_points=16
    )

    # 4. Навчаємо модель XGBoost.
    model = train_xgboost_classifier(features, sampled_labels)

    # 5. Класифікуємо кожен піксель нового зображення.
    resulting_mask = classify_entire_image(
        rgb_image, model, radius=2, n_points=16, class_to_color=class_to_color, expected_size=features.shape[1]
    )

    # 6. Зберігаємо фінальну класифікаційну маску.
    resulting_mask_rgb = cv2.cvtColor(resulting_mask, cv2.COLOR_BGR2RGB)
    cv2.imwrite("Images/temp/result_colored_mask_lbp.png", resulting_mask_rgb)
    plt.imshow(resulting_mask_rgb)
    plt.show()
```

---

## Конфігурування класифікатора

1. **`pixel_sample_percentage` для навчання:**
   - Змінюйте цей параметр, щоб контролювати кількість прикладів для навчання.
   - При дисбалансі класів рекомендую збільшити до `0.6–0.8`.

2. **`radius` і `n_points` для LBP:**
   - **Малі значення (`radius=1`, `n_points=8`)** — для дрібних текстур.
   - **Великі значення (`radius=3–5`, `n_points=16–24`)** — для більш обширних текстур.

3. **Параметри XGBoost:**
   - **`learning_rate (eta)`**:
     - Зменште до `0.05` для досконалого навчання.
   - **`max_depth`**:
     - Зменшіть до `4–6`, якщо модель перевчена.

4. **Гіперпараметри через `GridSearchCV`**:
   Використовуйте:
   ```python
   from sklearn.model_selection import GridSearchCV

   param_grid = {
       'n_estimators': [100, 200, 300],
       'max_depth': [4, 6, 8],
       'learning_rate': [0.05, 0.1, 0.2]
   }

   grid_search = GridSearchCV(xgb.XGBClassifier(objective='multi:softmax', num_class=4), param_grid, cv=3)
   grid_search.fit(features, sampled_labels)

   print("Best parameters:", grid_search.best_params_)
   ```

5. **Підгін розмірів ознак (інференс):**
   Якщо отримуєте помилки через розмір вектора, програма автоматично коригує це за допомогою `np.pad`.

---

## Висновок

Класифікатор:
- Ефективний для аналізу текстур і кольорів на зображенні.
- Може бути легко сконфігурований для різних задач, залежно від розміру маски, кількості класів, та рівня текстур.

### Рекомендації:
1. Провести **гіперпараметричний пошук `GridSearchCV`** для оптимізації XGBoost.
2. Спробувати різні **`radius` та `n_points`**, щоб знайти оптимальні налаштування для вашої текстури.